# 1. Quick tour

This notebook showcases a easiest example of training neural network potentials with PiNN

In [1]:
import os, warnings
import tensorflow as tf
from glob import glob
from ase.collections import g2
from pinn.datasets.qm9 import load_QM9_dataset
from pinn.models import potential_model
from pinn.calculator import PiNN_calc
# CPU is used for documentation generation, feel free to use your GPU!
os.environ['CUDA_VISIBLE_DEVICES'] = '' 
# We heavily use indexed slices to do sparse summations,
# which cause tensorflow to complain, 
# we believe it's safe to ignore this warning.
index_warning = 'Converting sparse IndexedSlices'
warnings.filterwarnings('ignore', index_warning)

## Getting the dataset

PiNN adapted tensorflow's dataset API to handel different datasets.

For this and the following notebooks the QM9 dataset (https://doi.org/10.6084/m9.figshare.978904) is used.  
To follow the notebooks, download the dataset and change the directory accordingly.

The dataset will be automatically split into subsets according to the split_ratio.  
Note that to use the dataset with the estimator, the datasets should be a function, instead of a dataset object.

In [2]:
filelist = glob('/home/yunqi/datasets/QM9/dsgdb9nsd/*.xyz')
dataset = lambda: load_QM9_dataset(filelist, split_ratio={'train':8, 'test':2})
train = lambda: dataset()['train'].repeat().shuffle(1000).batch(100)
test = lambda: dataset()['test'].repeat().batch(100)

## Defining the model
In PiNN, models are defined at two levels: models and networks. 
- A model (model_fn) defines the target, loss and training details of the network
- A network defines the structure of the neural network  

In this example, we will use the potential model, and the PiNN network.

The configuration of a model is stored in a nested json-like structure. 

```Python
{'model_dir': 'training_models/PiNN_QM9',
 'network': 'pinn_network',
 'netparam': {'depth': 4,
              'rc':4.0,
              'atom_types':[1,6,7,8,9]},
 'train':{
     'learning_rate': 3e-4,
     'regularization': 'clip',
     ...}}
```
A detailed description of the configuration **is NOT written yet**.

The easiest way to specify the network function is with its name.  
You can also provide a custom function here.

In [3]:
params = {'model_dir': '/tmp/PiNN_QM9',
          'network': 'pinn_network',
          'netparam': {},
          'train': {'learning_rate': 1e-3}}
model = potential_model(params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/PiNN_QM9', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1d26554940>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Configuring the training process
The defined model is indeed a tf.Estimator object, thus, the training can be easily controlled

In [4]:
train_spec = tf.estimator.TrainSpec(input_fn=train, max_steps=1000)
eval_spec = tf.estimator.EvalSpec(input_fn=test, steps=100)

## Train and evaluate

In [5]:
tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/PiNN_QM9/model.ckpt.
INFO:tensorflow:loss = 168317.23, step = 1
INFO:tensorflow:global_step/sec: 4.99999
INFO:tensorflow:loss = 6957.734, step = 101 (20.003 sec)
INFO:tensorflow:global_step/sec: 5.21264
INFO:tensorflow:loss = 6657.774, step = 201 (19.184 sec)
INFO:tensorflow:global_step/sec: 5.2076
INFO:tensorflow:loss = 1785.6746, step = 301 (19.202 sec)
INFO:tenso

({'ENG_MAE': 12.303868,
  'ENG_RMSE': 21.483816,
  'loss': 461.55426,
  'global_step': 1000},
 [])

## Using the model

The trained model can be used as a ASE calculator.

In [6]:
calc = PiNN_calc(model)
atoms = g2['C2H4']
atoms.set_calculator(calc)
atoms.get_forces(), atoms.get_potential_energy()

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/PiNN_QM9/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


(array([[-0.0000000e+00,  2.8312206e-06, -3.9432991e+00],
        [-0.0000000e+00,  7.1525574e-07,  3.9432998e+00],
        [-0.0000000e+00,  1.5139805e+01,  1.2248228e+01],
        [-0.0000000e+00, -1.5139809e+01,  1.2248232e+01],
        [-0.0000000e+00,  1.5139807e+01, -1.2248231e+01],
        [-0.0000000e+00, -1.5139807e+01, -1.2248231e+01]], dtype=float32),
 -94.6017074584961)

## Conclusion

You've have trained your first PiNN model, though the accuracy is not so satisfying
(RMSE=21 Hartree!). Also, the training speed is slow as it's limited by the IO and 
pre-processing of data.  

We will show in following notebooks that:
- Proper scaling of the energy will improve the accuracy of the model
- The training speed can be enhancing by caching and pre-processing the data